## Model Comparison Jupyter notebook

This notebook will load nets as defined int the imported python modules with certain preformance characteristics for comparison.
This file will also contain the tensorboard that helps visualize the models accuracy over time and the training process.

Components:
- initialize training parameters and fetch dataset (including RotMNIST dataset)
- compare model attributes (such as total parameters and structure)
- define hyperparameters
- create tensorboard and set up preformance graphs
- compare model training and preformance under different conditions
- save models to files

In [8]:
### Imports for pytorch and dataset

import torch
from torch.autograd import Variable

import torchvision
from torchvision import datasets
import torchvision.transforms as transforms

from torch.utils.data import DataLoader
import matplotlib.pyplot as plt

import torch.nn as nn

from RotMNIST import RotMNIST

In [9]:
### Define dataloaders
### Instatiate RotMNIST and verify behaviour below with the dataloaders
dataset_rot = RotMNIST(
    root = 'data',
    download=True,
    train=True,
    transform=torchvision.transforms.Compose(
        [torchvision.transforms.Resize(32), torchvision.transforms.ToTensor()]
    ),
    rotation_mirroring=True
)

test_dataset_rot = RotMNIST(
    root = 'data',
    download=True,
    train=False,
    transform=torchvision.transforms.Compose(
        [torchvision.transforms.Resize(32), torchvision.transforms.ToTensor()]
    ),
    rotation_mirroring=True
)

dataset_upright = RotMNIST(
    root = 'data',
    download=True,
    train=True,
    transform=torchvision.transforms.Compose(
        [torchvision.transforms.Resize(32), torchvision.transforms.ToTensor()]
    ),
    rotation_mirroring=False,
)

In [10]:
### Import different networks from python files
# TODO: Uncomment other networks and import
from p4m_conv import P4MNet
# from p4_conv import P4Net
# from z2_conv import ConvNet

p4m_net = P4MNet()
# p4_net = P4Net()
# conv_net = ConvNet()

p4m_total_params = sum(p.numel() for p in p4m_net.parameters() if p.requires_grad)
# p4_total_params = sum(p.numel() for p in p4_net.parameters() if p.requires_grad)
# z2_total_params = sum(p.numel() for p in conv_net.parameters() if p.requires_grad)

print(p4m_net)
print("P4M Trainable Params: " + str(p4m_total_params) + '\n')

# print(p4_net)
# print("P4 Trainable Params: " + str(p4_total_params) + '\n')

# print(conv_net)
# print("Conv Trainable Params: " + str(z2_total_params) + '\n')

P4MNet(
  (conv1): P4MConvZ2()
  (conv2): P4MConvP4M()
  (conv3): P4MConvP4M()
  (fc1): Linear(in_features=32, out_features=10, bias=True)
  (fc2): Linear(in_features=10, out_features=10, bias=True)
)
P4M Trainable Params: 122244



In [11]:
### Hyperparameters
learning_rate = 0.001
batch_size = 64
epochs = 25

### Objectives/Loss fn
loss_fn = nn.CrossEntropyLoss()

### Dataloaders
train_dataloader_rot = DataLoader(dataset_rot, batch_size=batch_size, shuffle=True)
test_dataloader_rot = DataLoader(test_dataset_rot, batch_size=batch_size, shuffle=True)
train_dataloader_upright = DataLoader(dataset_upright, batch_size=batch_size, shuffle=True)

In [12]:
### Tensorboard helpers
import matplotlib.pyplot as plt
import numpy as np

# helper function to show an image (copied from https://pytorch.org/tutorials/intermediate/tensorboard_tutorial.html)
# (used in the `plot_classes_preds` function below)
def imshow(img):
    img = img.mean(dim=0)
    img = img / 2 + 0.5     # unnormalize
    npimg = img.numpy()
    # plt.imshow(npimg, cmap="Greys")

### Tensorboard
import torch.utils.tensorboard
from torch.utils.tensorboard import SummaryWriter
writer = SummaryWriter('runs/p4m_mnist_1')

# Get grid of training images
dataiter = iter(train_dataloader_upright)
images, labels = dataiter.next()
img_grid = torchvision.utils.make_grid(images)
imshow(img_grid)

# To tensorboard
writer.add_image('Training Batch', img_grid)

writer.add_graph(p4m_net, images)
writer.close()

classes = ('0', '1', '2', '3', '4', '5', '6', '7', '8', '9')
# helper function
def select_n_random(data, labels, n=100):
    '''
    Selects n random datapoints and their corresponding labels from a dataset
    '''
    assert len(data) == len(labels)

    perm = torch.randperm(len(data))
    return data[perm][:n], labels[perm][:n]

# select random images and their target indices
images, labels = select_n_random(dataset_upright.data, dataset_upright.targets)

# get the class labels for each image
class_labels = [classes[lab] for lab in labels]

# log embeddings
features = images.view(-1, 28 * 28)
writer.add_embedding(features,
                    metadata=class_labels,
                    label_img=images.unsqueeze(1))
writer.close()

# helper functions

def images_to_probs(net, images):
    '''
    Generates predictions and corresponding probabilities from a trained
    network and a list of images
    '''
    output = net(images)
    # convert output probabilities to predicted class
    _, preds_tensor = torch.max(output, 1)
    preds = np.squeeze(preds_tensor.numpy())
    return preds, [torch.nn.functional.softmax(el, dim=0)[i].item() for i, el in zip(preds, output)]


def plot_classes_preds(net, images, labels):
    '''
    Generates matplotlib Figure using a trained network, along with images
    and labels from a batch, that shows the network's top prediction along
    with its probability, alongside the actual label, coloring this
    information based on whether the prediction was correct or not.
    Uses the "images_to_probs" function.
    '''
    preds, probs = images_to_probs(net, images)
    # plot the images in the batch, along with predicted and true labels
    fig = plt.figure(figsize=(12, 48))
    for idx in np.arange(4):
        ax = fig.add_subplot(1, 4, idx+1, xticks=[], yticks=[])
        imshow(images[idx])
        ax.set_title("{0}, {1:.1f}%\n(label: {2})".format(
            classes[preds[idx]],
            probs[idx] * 100.0,
            classes[labels[idx]]),
                    color=("green" if preds[idx]==labels[idx].item() else "red"))
    return fig


In [13]:
def train_loop(dataloader, model, loss_fn, optimizer):
    running_loss = 0.0
    size = len(dataloader.dataset)
    for batch, (X, y) in enumerate(dataloader):
        model.to(device)
        # Compute prediction and loss for backprop
        pred = model(X.to(device))
        loss = loss_fn(pred, y.to(device))

        # Backpropagation by setting grad to zero, calculating using backprop engine and stepping (using learning rate)
        optimizer.zero_grad()
        loss.backward()
        optimizer.step()

        running_loss += loss.item()

        if batch % 100 == 99:
            loss, current = loss.item(), batch * len(X)
            print(f"loss: {loss:>7f}  [{current:>5d}/{size:>5d}]")

            writer.add_scalar('training loss', running_loss / 100, cur_epoch * len(dataloader) + batch)
            writer.add_figure('predictions vs actuals', plot_classes_preds(model.to('cpu'), X, labels), global_step=cur_epoch * len(dataloader) + batch)
            running_loss = 0.0

def test_loop(dataloader, model, loss_fn):
    model.to(device)
    size = len(dataloader.dataset)
    test_loss, correct = 0, 0
    
    # No gradient on training data (faster computation and no optimization happening here anyway)
    with torch.no_grad():
        for X, y in dataloader:
            pred = model(X.to(device))
            test_loss += loss_fn(pred, y.to(device)).item()
            correct += (pred.argmax(1) == y.to(device)).type(torch.float).sum().item()

    test_loss /= size
    correct /= size
    
    print(f"Test Error: \n Accuracy: {(100*correct):>0.1f}%, Avg loss: {test_loss:>8f} \n")

In [14]:
### Upright training loop for p4m_net
optimizer = torch.optim.SGD(p4m_net.parameters(), lr=learning_rate, momentum=0.9)
device = torch.device("cuda:0" if torch.cuda.is_available() else "cpu")
for t in range(epochs):
    cur_epoch = t
    print(f"Epoch {t+1}\n-------------------------------")
    train_loop(train_dataloader_upright, p4m_net, loss_fn, optimizer)
    test_loop(test_dataloader_rot, p4m_net, loss_fn)
print('Finished Training P4M Net')

Epoch 1
-------------------------------
loss: 2.278352  [ 6336/60000]
loss: 1.997343  [12736/60000]
loss: 1.132336  [19136/60000]
loss: 0.958909  [25536/60000]
loss: 0.654748  [31936/60000]
loss: 0.648335  [38336/60000]
loss: 0.760683  [44736/60000]
loss: 0.709569  [51136/60000]
loss: 0.501799  [57536/60000]
Test Error: 
 Accuracy: 86.7%, Avg loss: 0.006629 

Epoch 2
-------------------------------
loss: 0.284434  [ 6336/60000]
loss: 0.339571  [12736/60000]
loss: 0.398244  [19136/60000]
loss: 0.274706  [25536/60000]
loss: 0.285697  [31936/60000]
loss: 0.118139  [38336/60000]
loss: 0.202080  [44736/60000]
loss: 0.290453  [51136/60000]
loss: 0.212414  [57536/60000]
Test Error: 
 Accuracy: 93.2%, Avg loss: 0.003547 

Epoch 3
-------------------------------
loss: 0.363411  [ 6336/60000]
loss: 0.226567  [12736/60000]
loss: 0.298706  [19136/60000]
loss: 0.094547  [25536/60000]


KeyboardInterrupt: 

### Model Comparisons

There are a few comparisons between the models to be made here. Here is a list of the following that I log
- Model accuracy on 10000 test images
- Model accuracy per class

In [ ]:
# ### Upright training loop for p4_net
# optimizer = torch.optim.SGD(p4_net.parameters(), lr=learning_rate, momentum=0.9)
# for t in range(epochs):
#     print(f"Epoch {t+1}\n-------------------------------")
#     train_loop(train_dataloader_upright, p4_net, loss_fn, optimizer)
#     test_loop(test_dataloader_rot, p4_net, loss_fn)
# print('Finished Training P4 Net')

In [ ]:
# ### Upright training loop for conv_net
# optimizer = torch.optim.SGD(conv_net.parameters(), lr=learning_rate, momentum=0.9)
# for t in range(epochs):
#     print(f"Epoch {t+1}\n-------------------------------")
#     train_loop(train_dataloader_upright, conv_net, loss_fn, optimizer)
#     test_loop(test_dataloader_rot, conv_net, loss_fn)
# print('Finished Training Conv Net')

In [ ]:
correct = 0
total = 0
# since we're not training, we don't need to calculate the gradients for our outputs
with torch.no_grad():
    for data in test_dataloader_rot:
        images, labels = data[0].to(device), data[1].to(device)
        # calculate outputs by running images through the network
        outputs = p4m_net(images)
        # the class with the highest energy is what we choose as prediction
        _, predicted = torch.max(outputs.data, 1)
        total += labels.size(0)
        correct += (predicted == labels.to(device)).sum().item()

print('Accuracy of the network on the 10000 test images: %f %%' % (
    100.0 * correct / total))

Accuracy of the network on the 10000 test images: 95.100000 %


In [ ]:
# prepare to count predictions for each class
correct_pred = {num : 0 for num in range(0, 10)}
total_pred = {num : 0 for num in range(0, 10)}

# again no gradients needed
with torch.no_grad():
    for data in test_dataloader_rot:
        images, labels = data[0].to(device), data[1].to(device)
        outputs = p4m_net(images.to(device))
        _, predictions = torch.max(outputs, 1)
        # collect the correct predictions for each class
        for label, prediction in zip(labels, predictions):
            if label == prediction:
                correct_pred[label.item()] += 1
            total_pred[label.item()] += 1


# print accuracy for each class
for classname, correct_count in correct_pred.items():
    accuracy = 100 * float(correct_count) / total_pred[classname]
    print("Accuracy for num {} is: {:.1f} %".format(classname,
                                                   accuracy))

Accuracy for num 0 is: 98.6 %
Accuracy for num 1 is: 98.6 %
Accuracy for num 2 is: 89.2 %
Accuracy for num 3 is: 95.5 %
Accuracy for num 4 is: 95.8 %
Accuracy for num 5 is: 92.3 %
Accuracy for num 6 is: 91.6 %
Accuracy for num 7 is: 98.1 %
Accuracy for num 8 is: 94.7 %
Accuracy for num 9 is: 95.8 %


In [ ]:
### Imports
from RotMNIST import RotMNIST

### New instance of CNN for testing on MNIST and RotMNIST
net = Net()
net.to(device)

### Hyperparameters
epochs = 10
learning_rate = 0.001
batch_size = 64


# Optimizers and Objectives
loss_fn = nn.CrossEntropyLoss()
optimizer = optim.SGD(net.parameters(), lr=learning_rate, momentum=0.9)

### Datasets and Dataloaders
transform = transforms.Compose(
    [transforms.ToTensor(),
     transforms.Resize((32,32))]
     )
dataset_rot = RotMNIST(
    root = 'data',
    download=True,
    train=True,
    transform=transform,
    rotation_mirroring=True
)

test_dataset_rot = RotMNIST(
    root = 'data',
    download=True,
    train=False,
    transform=transform,
    rotation_mirroring=True
)

dataset_upright = RotMNIST(
    root = 'data',
    download=True,
    train=True,
    transform=transform,
    rotation_mirroring=False,
)

### Instantiate dataloader for RotMNIST and get batches
train_dataloader_rot = torch.utils.data.DataLoader(dataset_rot, batch_size=batch_size, shuffle=True)
test_dataloader_rot = torch.utils.data.DataLoader(test_dataset_rot, batch_size=batch_size, shuffle=True)
train_dataloader_upright = torch.utils.data.DataLoader(dataset_upright, batch_size=batch_size, shuffle=True)

NameError: name 'Net' is not defined

In [ ]:
for t in range(epochs):
    print(f"Epoch {t+1}\n-------------------------------")
    train_loop(train_dataloader_rot, net, loss_fn, optimizer)
    test_loop(test_dataloader_rot, net, loss_fn)
print('Finished Training')

Epoch 1
-------------------------------
loss: 2.305217  [    0/60000]
loss: 2.296696  [ 6400/60000]
loss: 2.293112  [12800/60000]
loss: 2.294113  [19200/60000]
loss: 2.295065  [25600/60000]
loss: 2.278071  [32000/60000]
loss: 2.265457  [38400/60000]
loss: 2.223956  [44800/60000]
loss: 2.120625  [51200/60000]
loss: 1.505598  [57600/60000]
Test Error: 
 Accuracy: 27.4%, Avg loss: 0.036824 

Epoch 2
-------------------------------
loss: 1.200164  [    0/60000]
loss: 0.745467  [ 6400/60000]
loss: 0.519635  [12800/60000]
loss: 0.599424  [19200/60000]
loss: 0.524786  [25600/60000]
loss: 0.370485  [32000/60000]
loss: 0.350423  [38400/60000]
loss: 0.361508  [44800/60000]
loss: 0.539175  [51200/60000]
loss: 0.148360  [57600/60000]
Test Error: 
 Accuracy: 29.4%, Avg loss: 0.072286 

Epoch 3
-------------------------------
loss: 0.197427  [    0/60000]
loss: 0.248409  [ 6400/60000]
loss: 0.179247  [12800/60000]
loss: 0.343159  [19200/60000]
loss: 0.411106  [25600/60000]
loss: 0.194376  [32000/600